# 📖 AI-Powered Research & Writing with CrewAI & DeepSeek v3 on Fireworks  

In today's fast-paced world, the ability to **gather, synthesize, and present information efficiently** is crucial. This n\otebook demonstrates how to leverage **CrewAI** in combination with **Fireworks AI** to create an **automated research and writing workflow**.

By defining specialized AI **agents**, we can break down complex tasks into structured workflows where each agent plays a distinct role. This enables **automated information retrieval, summarization, and content generation**, making it easier to produce high-quality articles or reports with minimal manual effort.

## 🚀 What You’ll Learn
- **How to structure an AI-powered research team** using CrewAI.
- **How Fireworks AI enhances LLM inference** for research and writing tasks.
- **How to automate multi-step content creation workflows** with specialized agents.
- **How to integrate structured reasoning and function-calling for dynamic knowledge retrieval**.

Let's dive in and explore how CrewAI and Fireworks AI can **streamline your research and writing workflows!** ✨


In [ ]:
!pip3 install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29  > /dev/null

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.5.1+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12 12.5.82 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cuda-nvrtc-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-nvrtc-cu12 12.5.82 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cuda-runtime-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-runtime-cu12 12.5.82 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cudnn-cu12==9.1.0.70; platform_system 

## 🔧 Setting Up Your Environment  

Before diving into AI-powered research and writing, we need to configure our **environment**. This section ensures that all necessary **API keys** and environment variables are loaded correctly so that CrewAI and Fireworks AI can function smoothly.

We'll:
- **Load environment variables** from a `.env` file.
- **Retrieve API keys** for Fireworks AI and Serper (for web search).
- **Define utility functions** for managing API access and formatting responses.

By organizing these components upfront, we can seamlessly interact with **Fireworks AI** for inference and **Serper API** for real-time web search.


Your `.env` file should look something like:


```
# Fireworks AI API Key
OPENAI_API_KEY=your_fireworks_api_key_here

# Serper API Key (if needed for web search capabilities)
SERPER_API_KEY=your_serper_api_key_here

```



In [ ]:
import os
from dotenv import load_dotenv, find_dotenv

# Load environment variables from a .env file
def load_env():
    _ = load_dotenv(find_dotenv())

def get_openai_api_key():
    """Returns the API key for Fireworks AI, using OpenAI's expected variable name."""
    load_env()
    openai_api_key = os.getenv("OPENAI_API_KEY")
    return openai_api_key

def get_openai_api_base():
    """Returns the API base URL for Fireworks AI."""
    load_env()
    openai_api_base = os.getenv("OPENAI_API_BASE", "https://api.fireworks.ai/inference/v1")
    return openai_api_base

def get_serper_api_key():
    """Returns the API key for Serper."""
    load_env()
    serper_api_key = os.getenv("SERPER_API_KEY")
    return serper_api_key

# break line every 80 characters if line is longer than 80 characters
# don't break in the middle of a word
def pretty_print_result(result):
    parsed_result = []
    for line in result.split('\n'):
        if len(line) > 80:
            words = line.split(' ')
            new_line = ''
            for word in words:
                if len(new_line) + len(word) + 1 > 80:
                    parsed_result.append(new_line)
                    new_line = word
                else:
                    if new_line == '':
                        new_line = word
                    else:
                        new_line += ' ' + word
            parsed_result.append(new_line)
        else:
            parsed_result.append(line)
    return "\n".join(parsed_result)



## 🛠️ Building the Research & Writing AI Crew  

Now that our environment is set up, we can start constructing our **AI-powered research team** using CrewAI. CrewAI enables us to define **agents**, assign them specific **tasks**, and coordinate their workflow to complete complex research and writing objectives efficiently.

### 🔹 What We'll Do in This Section:
- **Define AI Agents** with specialized roles (e.g., researcher, writer).
- **Assign Tasks** to each agent, ensuring a clear workflow.
- **Assemble a Crew** that collaborates to produce high-quality content.

This structured approach allows us to **combine multiple AI capabilities**, making them work together as an autonomous research team.


In [ ]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from crewai import Agent, Task, Crew

## 🔑 Configuring API Access for Fireworks AI  

Before deploying our AI-powered research team, we need to **configure access to Fireworks AI** by setting up API credentials. This ensures that our agents can interact with the **DeepSeek V3** model for reasoning, structured outputs, and function calling.

### 🔹 What This Code Does:
- **Retrieves API Credentials** from environment variables.
- **Configures OpenAI-Compatible Settings** for seamless integration with Fireworks.
- **Defines the Default Model** (`DeepSeek V3`) to be used for CrewAI tasks.

By setting these environment variables, our agents can interact with Fireworks AI in a way that’s compatible with OpenAI’s SDK, making it easier to integrate with existing workflows.


In [ ]:
import os

# Load API key and base URL
openai_api_key = get_openai_api_key()
openai_api_base = get_openai_api_base()

# Set environment variables for OpenAI SDK compatibility with Fireworks
os.environ["OPENAI_API_KEY"] = openai_api_key
os.environ["OPENAI_API_BASE"] = openai_api_base
os.environ["OPENAI_MODEL_NAME"] = "accounts/fireworks/models/deepseek-v3"


# 🛠️ Creating Your Agents for Content Creation  

To **automate the research and writing process**, we define three AI-powered agents with distinct roles and objectives:  

1️⃣ **Content Planner** – Researches and outlines key points on the given topic.  
2️⃣ **Content Writer** – Drafts a well-structured, opinion-backed article.  
3️⃣ **Editor** – Reviews and refines the content for clarity, balance, and adherence to style guidelines.  

Each agent is assigned a **role, goal, and backstory**, which enhances their performance by enabling effective **role-playing**—a technique that has been observed to improve LLM output quality.  

Now, let’s define these agents! 👇  


### 📝 Defining the **Content Planner Agent**  

The **Content Planner Agent** structures and organizes key information for an article, ensuring it is **engaging and factually accurate**.

#### 🔹 Responsibilities:
- **Plans** the article structure based on `{topic}`.  
- **Gathers** relevant, educational, and decision-informing insights.  
- **Lays the foundation** for the Content Writer to craft the final piece.  

This agent works independently, **does not delegate tasks**, and provides **detailed logs** for transparency.


In [ ]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    allow_delegation=False,
	  verbose=True,
    openai_api_key=get_openai_api_key()
)

### ✍️ Defining the **Content Writer Agent**  

The **Content Writer Agent** crafts an **insightful and factually accurate** opinion piece based on the Content Planner's outline.

#### 🔹 Responsibilities:
- **Writes** a structured opinion piece following the provided outline.  
- **Incorporates** relevant context and supporting details.  
- **Differentiates** between factual statements and personal opinions.  

This agent **does not delegate tasks** and maintains **detailed logs** for transparency.


In [ ]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    verbose=True,
    openai_api_key=get_openai_api_key()
)

### 📝 Defining the **Editor Agent**  

The **Editor Agent** refines the blog post to align with **journalistic best practices** and the **organization's writing style**.

#### 🔹 Responsibilities:
- **Reviews** the Content Writer's draft for clarity, structure, and tone.  
- **Ensures** balanced viewpoints and fact-checks assertions.  
- **Avoids** major controversial topics unless necessary.  

This agent **does not delegate tasks** and keeps **detailed logs** for transparency.


In [ ]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True,
    openai_api_key=get_openai_api_key()
)

# 📝 Creating Tasks for Each Agent  

Now that we have our **Content Planner, Writer, and Editor**, we need to assign them specific **tasks** to ensure a smooth workflow.  

Each task should clearly define:  
- **The objective** – What the agent needs to accomplish.  
- **The agent responsible** – Which agent is best suited for this task.  
- **The expected output** – What kind of response or data the agent should generate.  

By structuring tasks effectively, we ensure that each agent operates efficiently and contributes to a well-researched and well-written final piece.  

Let’s create and assign the tasks! 👇  


## 🛠️ Creating Tasks for the Content Planner  

The **Content Planner** is responsible for setting the foundation of the article by researching the latest trends, identifying the target audience, and structuring the article.  

To ensure the planner generates a well-structured content outline, we define a **task** that includes:  
- Researching **key players and trends** in the given topic.  
- Understanding the **target audience** and their pain points.  
- Structuring an **SEO-friendly content outline** with relevant keywords.  

This structured approach ensures that the **Writer** has a solid roadmap to craft the article. Let’s define the **Content Planner’s task** below. 👇  


In [ ]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)

## ✍️ Assigning the Writing Task  

With a structured content plan in place, the **Content Writer** takes over to transform the outline into a **compelling, well-written blog post**.  

The writing task ensures that:  
- The **content plan** is fully utilized to create an engaging article.  
- **SEO keywords** are integrated seamlessly into the text.  
- The structure includes a **clear introduction, insightful body, and strong conclusion**.  
- The post aligns with the **brand's tone and style guide** while maintaining readability.  

Let’s define the **Content Writer’s task** below. 👇  


In [ ]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
)

## 📝 Refining the Blog Post with the Editor  

Once the **Content Writer** completes the article, the **Editor** steps in to **review and refine** it. The editor's role is to:  

- **Ensure grammatical accuracy** and readability.  
- **Maintain alignment** with the brand's voice and editorial standards.  
- **Enhance clarity** while preserving the original intent of the content.  

This final step polishes the blog post to make it **publication-ready**. Let’s define the **Editor’s task** below. 👇  


In [ ]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

# 🚀 Bringing It All Together: Creating the Crew  

Now that we've defined our **agents** and their **tasks**, we need to assemble them into a **Crew**. The **Crew** orchestrates the workflow, ensuring that each agent completes its assigned role before passing the results to the next stage.  

Here’s what happens:  
1. The **Content Planner** researches and creates a structured content plan.  
2. The **Content Writer** drafts the blog post based on the planner’s outline.  
3. The **Editor** refines the article, ensuring clarity, correctness, and brand alignment.  

With all agents and tasks in place, let’s initialize our **Crew** and get started! 🚀  


In [ ]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=2
)

## 🎬 Kicking Off the Workflow  

With our **Crew** fully assembled and all agents ready to execute their respective tasks, it's time to launch the workflow! 🚀  

By calling `kickoff()`, we initiate the **end-to-end content creation process**, starting from research and planning, followed by writing, and concluding with editing. The `inputs` dictionary provides the initial topic for the content generation.  

Let's see how our AI-powered **Crew** collaborates to produce a high-quality article on **Artificial Intelligence**!  


In [ ]:
result = crew.kickoff(inputs={"topic": "Artificial Intelligence"})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Artificial Intelligence.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
Thought: I now can give a great answer  
Final Answer:  

### Comprehensive Content Plan: Artificial Intelligence  

---

**1. Target Audience Analysis**  
**Primary Audience:**  
- Tech enthusiasts, developers, and data scientists who are actively engaged in AI research or implementation.  
- Business leaders and decision-makers exploring AI for organizational growth and efficiency.  
- General readers interested in understanding AI trends and their impact on society.  

**Interests and Pain Points:**  
- **Tech Enthusiasts/Developers:** Staying updated

## 📄 Displaying the Final Article  

Once the workflow is complete, the final **AI-generated blog post** is stored in `result`. To render it in a clean, readable format, we use `IPython.display.Markdown()`, which converts the raw markdown text into a properly formatted output.  

Let's review the structured and polished article produced by our **Crew**! 🚀  


In [ ]:
from IPython.display import Markdown
Markdown(result)

The blog post is now polished, grammatically correct, and aligned with the organization’s writing style, making it ready for publication.

## 🔄 Running the Workflow with a New Topic  

To explore how our **AI-driven research and writing workflow** adapts to different subjects, we can rerun the **Crew** with a new topic. By updating the input parameter, we instruct our agents to generate fresh insights and structured content on **DeepSeek R1**.  

Let's see how the system plans, writes, and refines a new article focused on **DeepSeek R1**! 🚀  


In [ ]:
topic = "DeepSeek R1"
result = crew.kickoff(inputs={"topic": topic})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on DeepSeek R1.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
Thought: I now can give a great answer  
Final Answer:  

**Content Plan Document: DeepSeek R1 Blog Article**  

---

### **1. Target Audience Analysis**  
**Primary Audience:**  
- Tech enthusiasts and early adopters interested in cutting-edge AI and robotics.  
- Professionals and researchers in the fields of AI, robotics, and automation.  
- Business leaders and decision-makers exploring innovative solutions for efficiency and productivity.  
- Investors and startups looking for emerging technologies with high growth potential.  

**Interests and Pain Points:**  

In [ ]:
Markdown(result)

```markdown
# DeepSeek R1: Revolutionizing AI and Robotics – What You Need to Know  

The world of AI and robotics is advancing at an unprecedented pace, and at the forefront of this revolution is **DeepSeek R1**. This groundbreaking AI-powered robotics system is not just another technological marvel—it’s a game-changer with the potential to transform industries and redefine how we interact with technology. Whether you’re a tech enthusiast, a business leader, or an investor, understanding DeepSeek R1 is crucial to staying ahead in this rapidly evolving landscape. In this article, we’ll explore what makes DeepSeek R1 unique, its real-world applications, and the impact it’s poised to have on the future of AI and robotics.  

## What is DeepSeek R1?  

At its core, **DeepSeek R1** is an AI-powered robotics system designed to push the boundaries of automation and intelligence. It combines cutting-edge AI algorithms with advanced hardware to deliver unmatched precision, efficiency, and adaptability. Unlike traditional robotics systems, DeepSeek R1 is built to seamlessly integrate into complex workflows, making it a versatile solution for a wide range of industries.  

One of its standout features is its ability to learn and adapt in real-time, thanks to its advanced machine learning capabilities. This allows DeepSeek R1 to handle tasks that were previously considered too complex or unpredictable for automation. Compared to existing robotics systems, DeepSeek R1 stands out for its innovation, scalability, and potential to drive significant improvements in productivity and cost-effectiveness.  

## Latest Trends and Developments  

The AI and robotics landscape is constantly evolving, and DeepSeek R1 is at the heart of some of the most exciting developments. Recently, the system has seen significant updates, including enhanced algorithms and expanded compatibility with various industry-standard platforms. These updates ensure that DeepSeek R1 remains at the cutting edge of technology.  

Moreover, DeepSeek R1 has garnered attention through strategic partnerships and collaborations with leading organizations in tech, healthcare, and manufacturing. These alliances not only validate its potential but also pave the way for broader adoption across industries. As AI continues to dominate tech trends in 2023, DeepSeek R1 is a prime example of how innovation is shaping the future of robotics.  

## Key Players and Industry Impact  

DeepSeek R1 is already making waves across multiple sectors. In **healthcare**, it’s being used to streamline surgical procedures and improve patient care. In **manufacturing**, it’s enhancing production lines by reducing errors and increasing efficiency. Logistics companies are leveraging its capabilities to optimize supply chains, while retailers are using it to improve customer experiences through personalized automation.  

Notable case studies highlight the tangible benefits of DeepSeek R1. For instance, a leading healthcare provider reported a 30% reduction in surgical errors after integrating the system into their operations. Similarly, a global manufacturing giant achieved a 20% increase in production efficiency. These success stories underscore the transformative potential of DeepSeek R1 and its ability to deliver measurable results.  

## Practical Applications  

The real-world applications of DeepSeek R1 are as diverse as they are impactful. In **healthcare**, it’s being used for precision surgeries, patient monitoring, and even drug discovery. In **manufacturing**, it’s automating repetitive tasks, improving quality control, and reducing downtime. Logistics companies are using it to optimize warehouse operations and streamline delivery processes.  

One particularly compelling example is its use in retail, where DeepSeek R1 is powering automated customer service systems. These systems can handle inquiries, process orders, and even provide personalized recommendations—all without human intervention. As the technology continues to evolve, the possibilities for its application are virtually limitless.  

## Benefits and Challenges  

The advantages of DeepSeek R1 are clear: it offers **cost savings**, **precision**, and **scalability**, making it an attractive investment for businesses. Its ability to learn and adapt ensures that it remains relevant even as industry demands change. However, like any advanced technology, it’s not without its challenges.  

One of the primary concerns is the **high initial cost**, which may be a barrier for smaller businesses. Additionally, the **technical complexity** of integrating and maintaining the system requires specialized expertise. To overcome these challenges, businesses can partner with experienced providers and explore financing options to make the investment more manageable.  

## The Future of DeepSeek R1  

Looking ahead, the future of DeepSeek R1 is incredibly promising. As AI and robotics continue to evolve, we can expect the system to become even more sophisticated and versatile. Market analysts predict significant growth in the adoption of AI-powered robotics, and DeepSeek R1 is well-positioned to lead this charge.  

For businesses and individuals, the opportunities to get involved are vast. Whether you’re looking to integrate DeepSeek R1 into your operations, invest in its development, or simply stay informed about the latest trends, now is the time to act. The future of AI and robotics is here, and DeepSeek R1 is at the forefront.  

## Call to Action  

Ready to explore how DeepSeek R1 can transform your business? Visit the official website to learn more about its features and applications. Stay ahead of the curve by subscribing to our newsletter for the latest updates on AI and robotics trends. Have questions or thoughts about DeepSeek R1? Share them in the comments below—we’d love to hear from you!  
```

# 🎯 Conclusion: Unlocking the Power of Fireworks with CrewAI  

Through this structured workflow, we've explored how **Fireworks AI** enables **DeepSeek R1 and V3** to generate structured reasoning outputs and leverage function calling for dynamic, real-time interactions. By integrating these capabilities with **CrewAI**, we've built an intelligent, automated content generation pipeline that can research, write, and refine articles efficiently.

### 🚀 Key Takeaways:
- **DeepSeek R1** is optimized for **structured reasoning**, making it ideal for logical, step-by-step responses.
- **DeepSeek V3** supports **function calling**, allowing it to interact with external tools and APIs dynamically.
- **Fireworks AI** provides **fast, serverless inference**, making it an ideal **inference provider** for AI-driven workflows.
- **CrewAI** allows for **multi-agent collaboration**, making it easy to orchestrate intelligent workflows using LLMs.

### 🔥 What’s Next?
Now that you’ve seen how **Fireworks AI** enhances LLM workflows, consider:
* ✅ **Experimenting** with different topics and refining agent roles.  
* ✅ **Integrating Fireworks inference** into your own CrewAI-powered projects.  
* ✅ **Exploring other structured outputs** with Fireworks' **JSON Mode & Grammar Mode**.  

Fireworks AI provides **best-in-class model hosting and optimization**—ensuring that your AI-powered applications run **fast, reliably, and at scale**.

Start building with **Fireworks AI + CrewAI** today! 🚀🎆  
